In [66]:
import pandas as pd
import numpy as np
from collections import Counter #Use to display most frequent object and its key

In [67]:
ls

export_file/          glasstire-datahack/   main_glasstire.ipynb


## 1) Obtain first df set on event_uid

In [68]:
#Import data
event_artists=pd.read_csv('glasstire-datahack/event_artists.csv')
events=pd.read_csv('glasstire-datahack/events.csv')
venues=pd.read_csv('glasstire-datahack/venues.csv')

In [69]:
#Create the new event_artists (because it is suck)
unique_events_with_artist=pd.DataFrame({})
unique_events=list(event_artists['event_uid'].unique())
num_artist=[]
artist_list=[]
for event in unique_events:
    foo=list(event_artists[event_artists['event_uid']==event]['artist_uid'])
    if len(foo)>0:
        artist_list.append(foo)
        num_artist.append(len(foo))
    else:
        print(0) #As you run it, this shows that each event in the table have at least one artist 
        artist_list.append([0])
        num_artist.append(0)

#There are some events that are not in this table
#Find them and append 0 to their corresponding artist_list column
diff=set(events['event_uid'])-set(unique_events)
print("Number of events that do not have any artist:",len(diff)) #Expected 9763-7476=2287
for i in diff:
    unique_events.append(i)
    artist_list.append(0)
    num_artist.append(0)

unique_events_with_artist['event_uid']=unique_events
unique_events_with_artist['artist_list']=artist_list
unique_events_with_artist['num_artist']=num_artist
print("Lenght of our new data set:",len(unique_events_with_artist))
unique_events_with_artist.sample(2)

Number of events that do not have any artist: 2287
Lenght of our new data set: 9763


,event_uid,artist_list,num_artist
5934,13430405,[2574801],1
4113,9297887,"[810052, 8308366, 11213028]",3


In [70]:
#Combine all the tabke we have so far together
dffoo=pd.DataFrame({})
df=pd.DataFrame({})
dffoo=events.merge(unique_events_with_artist,on=['event_uid'])
df=dffoo.merge(venues,on=['venue_uid'])
df['event_uid']=df['event_uid'].astype(str)
df['venue_uid']=df['venue_uid'].astype(str)

In [71]:
print(len(df))
df.sample(3)

9763


,event_uid,venue_uid,start_date,end_date,artist_list,num_artist,venue,coordinates,address,city,zip,url
9659,16762285,8781562,2013-01-27,2013-01-27,[7097677],1,Studio 101,"29.7752312857143,-95.3762571836735",1824 Spring Street,Houston,77007.0,springstreetstudios.info/2012/
3121,4907983,5031284,2016-04-09,2016-05-21,[13093806],1,Laura Rathe Fine Art,"29.7347594545455,-95.4202408181818",2707 Colquitt Street,Houston,77098.0,www.laurarathe.com
2542,13621969,6618085,2015-11-21,2016-02-14,"[170349, 489602, 963851, 980135, 4725480, 5309...",22,Contemporary Arts Museum Houston,"29.726739755102,-95.391116",5216 Montrose Boulevard,Houston,77006.0,www.camh.org


In [72]:
#Output first dataset
!rm export_file/combine_data.csv #Prevent duplication
df.to_csv('export_file/combine_data.csv')

## 2) Obtain second df set on artist_id

In [73]:
#Input event id, return venue name and start day
def venues_name(event_uid):
    
    data=events[events['event_uid']==event_uid]
    venue_uid=list(data['venue_uid'])[0]
    start_date=list(data['start_date'])[0]
    venue=list(venues[venues['venue_uid']==venue_uid]['venue'])[0]
    url=list(venues[venues['venue_uid']==venue_uid]['url'])[0]
    return (venue,start_date,url)

In [74]:
#Input artist id, output their start year and end year of event activity in houston
def num_year_activity(artist_uid):
    event_list=list(artists_num_events[artists_num_events['artist_uid']==artist_uid]['event_list'])[0]
    date_list=[]
    year_list=[]
    for event in event_list:
        date_list.append(venues_name(event)[1])
    
    for date in date_list:
        year_list.append(int(date[:4]))
    
    return (min(year_list),max(year_list))    

In [75]:
#Input a list, return the most frequent object and its key
def most_frequent(List): 
    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0]

In [76]:
#Make a new table for unique artists with all of the event they are in
event_list=[]
num_event_list=[]
venue_list=[]
for i in event_artists['artist_uid'].unique():
    events_foo=list(event_artists[event_artists['artist_uid']==i]['event_uid'])
    
    venue_list.append([venues_name(i)[0] for i in events_foo])
    event_list.append(events_foo)
    num_event_list.append(len(events_foo))
    
    
artists_num_events=pd.DataFrame({})

artists_num_events['artist_uid']=event_artists['artist_uid'].unique()
artists_num_events['event_list']=event_list
artists_num_events['num_event']=num_event_list
artists_num_events['fav_venue']= [most_frequent(i)[0] for i in venue_list]
artists_num_events['num_event_fav_venue']=[most_frequent(i)[1] for i in venue_list]
artists_num_events=artists_num_events.sort_values(by=['num_event'],ascending=False)

In [77]:
artists_num_events.head()

,artist_uid,event_list,num_event,fav_venue,num_event_fav_venue
60,4199480,"[95014, 521154, 709785, 792948, 819381, 911516...",48,Gallery 1724,6
58,2420053,"[95014, 1014354, 1844447, 2104017, 2257651, 27...",35,Art League Houston,4
171,5309938,"[194616, 203277, 508972, 1025367, 1042980, 121...",31,Optical Project,7
52,170349,"[88961, 535862, 1157840, 1298583, 1426629, 145...",29,Project Row Houses,8
420,5604161,"[453265, 570398, 830057, 1025367, 1133907, 258...",29,The Joanna Gallery,5


In [78]:
#Take about 1 min to run
start_active_year_list=[]
end_active_year_list=[]
count=0
for id_ in artists_num_events['artist_uid']:
    start_active_year_list.append(num_year_activity(id_)[0])
    end_active_year_list.append(num_year_activity(id_)[1])

In [79]:
#Adding new columns for our new data frame
event_per_year=np.array(artists_num_events['num_event'])/(np.array(end_active_year_list)-np.array(start_active_year_list)+1)
event_per_year=[round(i,2) for i in event_per_year]

artists_num_events['start_active_year']=start_active_year_list
artists_num_events['end_active_year']=end_active_year_list
artists_num_events['event_per_year']=event_per_year
artists_num_events['num_year_active']=np.array(end_active_year_list)-np.array(start_active_year_list)+1

artists_num_events.head()

,artist_uid,event_list,num_event,fav_venue,num_event_fav_venue,start_active_year,end_active_year,event_per_year,num_year_active
60,4199480,"[95014, 521154, 709785, 792948, 819381, 911516...",48,Gallery 1724,6,2007,2019,3.69,13
58,2420053,"[95014, 1014354, 1844447, 2104017, 2257651, 27...",35,Art League Houston,4,2007,2017,3.18,11
171,5309938,"[194616, 203277, 508972, 1025367, 1042980, 121...",31,Optical Project,7,2007,2015,3.44,9
52,170349,"[88961, 535862, 1157840, 1298583, 1426629, 145...",29,Project Row Houses,8,2012,2019,3.62,8
420,5604161,"[453265, 570398, 830057, 1025367, 1133907, 258...",29,The Joanna Gallery,5,2007,2019,2.23,13


In [80]:
# Export second file
!rm export_file/artists_num_events_new.csv #Prevent duplication
artists_num_events.to_csv('export_file/artists_num_events_new.csv')

#### Some data exploration on the event (Ignore this part)

In [81]:
foo=artists_num_events.sort_values(by=['event_per_year'],ascending=False)
foo[foo['end_active_year']==2019].head()

,artist_uid,event_list,num_event,fav_venue,num_event_fav_venue,start_active_year,end_active_year,event_per_year,num_year_active
60,4199480,"[95014, 521154, 709785, 792948, 819381, 911516...",48,Gallery 1724,6,2007,2019,3.69,13
52,170349,"[88961, 535862, 1157840, 1298583, 1426629, 145...",29,Project Row Houses,8,2012,2019,3.62,8
246,9258618,"[310956, 556662, 698913, 1439349, 2384852, 257...",23,Lawndale Art Center,5,2011,2019,2.56,9
420,5604161,"[453265, 570398, 830057, 1025367, 1133907, 258...",29,The Joanna Gallery,5,2007,2019,2.23,13
500,3979934,"[543861, 819381, 2193303, 2377976, 2964982, 35...",22,Lawndale Art Center,3,2010,2019,2.20,10


In [82]:
artists_num_events.head()

,artist_uid,event_list,num_event,fav_venue,num_event_fav_venue,start_active_year,end_active_year,event_per_year,num_year_active
60,4199480,"[95014, 521154, 709785, 792948, 819381, 911516...",48,Gallery 1724,6,2007,2019,3.69,13
58,2420053,"[95014, 1014354, 1844447, 2104017, 2257651, 27...",35,Art League Houston,4,2007,2017,3.18,11
171,5309938,"[194616, 203277, 508972, 1025367, 1042980, 121...",31,Optical Project,7,2007,2015,3.44,9
52,170349,"[88961, 535862, 1157840, 1298583, 1426629, 145...",29,Project Row Houses,8,2012,2019,3.62,8
420,5604161,"[453265, 570398, 830057, 1025367, 1133907, 258...",29,The Joanna Gallery,5,2007,2019,2.23,13


In [83]:
#Check all the event + venue associate with Emily Sloan 
a=list(artists_num_events[artists_num_events['artist_uid']==4199480]['event_list'])[0]
for i in a:
    print(venues_name(i))

('Williams Tower Gallery', '2011-06-16', nan)
('Orange Show Center for Visionary Art', '2011-10-28', 'www.orangeshow.org')
('labotanica', '2010-09-25', 'labotanica.org')
('Darke Gallery', '2012-08-11', 'www.darkegallery.com')
('Art League Houston', '2012-11-16', 'www.artleaguehouston.org')
('BLUEorange Contemporary', '2013-07-19', 'blueorangehouston.com')
('Blaffer Art Museum (UH)', '2009-04-10', 'blafferartmuseum.org')
('BLUEorange Contemporary', '2013-08-02', 'blueorangehouston.com')
('Lawndale Art Center', '2009-08-21', 'www.lawndaleartcenter.org')
('Houston Center for Photography', '2013-11-22', 'www.hcponline.org')
('Intexure', '2007-03-31', nan)
('Discovery Green', '2011-09-01', 'www.discoverygreen.com')
('Darke Gallery', '2012-08-18', 'www.darkegallery.com')
('Gallery 1724', '2012-03-11', 'www.gallery1724.blogspot.com')
('Art League Houston', '2011-08-25', 'www.artleaguehouston.org')
('14 Pews', '2011-06-18', '14pews.org')
('Gallery 1724', '2010-02-06', 'www.gallery1724.blogspot